In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import Lasso

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
train = pd.read_csv('train_1.csv')

In [5]:
train.drop(columns=['Unnamed: 0', 'patient_id'], axis=1, inplace=True)

In [6]:
train

,patient_zip3,patient_age,population,density,age_median,age_under_10,age_10_to_19,age_20s,age_30s,age_40s,age_50s,age_60s,age_70s,age_over_80,male,female,married,divorced,never_married,widowed,family_size,family_dual_income,income_household_median,income_household_under_5,income_household_5_to_10,income_household_10_to_15,income_household_15_to_20,income_household_20_to_25,income_household_25_to_35,income_household_35_to_50,income_household_50_to_75,income_household_75_to_100,income_household_100_to_150,income_household_150_over,income_household_six_figure,income_individual_median,home_ownership,housing_units,home_value,rent_median,rent_burden,education_less_highschool,education_highschool,education_some_college,education_bachelors,education_graduate,education_college_or_above,education_stem_degree,labor_force_participation,unemployment_rate,self_employed,farmer,race_white,race_black,race_asian,race_native,race_pacific,race_other,race_multiple,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02,DiagPeriodL90D,patient_race_Asian,patient_race_Black,patient_race_Hispanic,patient_race_Other,patient_race_White,payer_type_COMMERCIAL,payer_type_MEDICAID,payer_type_MEDICARE ADVANTAGE,Region_Midwest,Region_Northeast,Region_South,Region_West,Division_East North Central,Division_East South Central,Division_Middle Atlantic,Division_Mountain,Division_Pacific,Division_South Atlantic,Division_West North Central,Division_West South Central,patient_state_AK,patient_state_AL,patient_state_AR,patient_state_AZ,patient_state_CA,patient_state_CO,patient_state_DC,patient_state_DE,patient_state_FL,patient_state_GA,patient_state_HI,patient_state_IA,patient_state_ID,patient_state_IL,patient_state_IN,patient_state_KS,patient_state_KY,patient_state_LA,patient_state_MD,patient_state_MI,patient_state_MN,patient_state_MO,patient_state_MS,patient_state_MT,patient_state_NC,patient_state_ND,patient_state_NE,patient_state_NM,patient_state_NV,patient_state_NY,patient_state_OH,patient_state_OK,patient_state_OR,patient_state_PA,patient_state_SC,patient_state_SD,patient_state_TN,patient_state_TX,patient_state_UT,patient_state_VA,patient_state_WA,patient_state_WI,patient_state_WV,patient_state_WY,patient_gender_F,breast_cancer_diagnosis_code_1741,breast_cancer_diagnosis_code_1742,breast_cancer_diagnosis_code_1743,breast_cancer_diagnosis_code_1744,breast_cancer_diagnosis_code_1745,breast_cancer_diagnosis_code_1746,breast_cancer_diagnosis_code_1748,breast_cancer_diagnosis_code_1749,breast_cancer_diagnosis_code_1759,breast_cancer_diagnosis_code_19881,breast_cancer_diagnosis_code_C50,breast_cancer_diagnosis_code_C5001,breast_cancer_diagnosis_code_C50011,breast_cancer_diagnosis_code_C50012,breast_cancer_diagnosis_code_C50019,breast_cancer_diagnosis_code_C50021,breast_cancer_diagnosis_code_C5011,breast_cancer_diagnosis_code_C50111,breast_cancer_diagnosis_code_C50112,breast_cancer_diagnosis_code_C50119,breast_cancer_diagnosis_code_C5021,breast_cancer_diagnosis_code_C50211,breast_cancer_diagnosis_code_C50212,breast_cancer_diagnosis_code_C50219,breast_cancer_diagnosis_code_C5031,breast_cancer_diagnosis_code_C50311,breast_cancer_diagnosis_code_C50312,breast_cancer_diagnosis_code_C50319,breast_cancer_diagnosis_code_C5041,breast_cancer_diagnosis_code_C50411,breast_cancer_diagnosis_code_C50412,breast_cancer_diagnosis_code_C50419,breast_cancer_diagnosis_code_C50421,breast_cancer_diagnosis_code_C5051,breast_cancer_diagnosis_code_C50511,breast_cancer_diagnosis_code_C50512,breast_cancer_diagnosis_code_C50519,breast_cancer_diagnosis_code_C50611,breast_cancer_diagnosis_code_C50612,breast_cancer_diagnosis_code_C50619,breast_cancer_diagnosis_code_C5081,breast_cancer_diagnosis_code_C50811,breast_cancer_diagnosis_code_C50812,breast_cancer_diagnosis_code_C50819,breast_cancer_diagnosis_code_C509,breast_cancer_diagnosis_code_C5091,breast_cancer_diagnosis_code_C50911,breast_cancer_diagnosis_code_C50912,breast_cancer_diagnosis_code_C50919,breast_cance

In [7]:
train.isna().sum().sort_values(ascending=False)

patient_zip3                             0
breast_cancer_diagnosis_code_1759        0
breast_cancer_diagnosis_code_C50         0
breast_cancer_diagnosis_code_C5001       0
breast_cancer_diagnosis_code_C50011      0
                                        ..
Region_Northeast                         0
Region_South                             0
Region_West                              0
Division_East North Central              0
metastatic_cancer_diagnosis_code_C799    0
Length: 228, dtype: int64

In [8]:
target = 'DiagPeriodL90D'

In [9]:
locations = [column for column in train.columns if 'patient_state' in column or 'Region' in column  or 'Division' in column]
locations = train[locations]

In [10]:
age = [column for column in train.columns if 'age' in column]
age = train[age]

In [11]:
race = [column for column in train.columns if 'race' in column]
race = train[race]

In [12]:
education = [column for column in train.columns if 'education' in column]
education = train[education]

In [13]:
income = [column for column in train.columns if 'income' in column]
income = train[income]

In [14]:
rent = [column for column in train.columns if 'rent' in column]
rent = train[rent]

In [15]:
social_status = [column for column in train.columns if 'married' in column or 'divorced' in column or 'widowed' in column]
social_status = train[social_status]

In [16]:
empl_status = [column for column in train.columns if 'labor' in column or 'employ' in column]
empl_status = train[empl_status]

In [17]:
home_status = [column for column in train.columns if 'home' in column or 'housing' in column]
home_status = train[home_status]

In [18]:
health_status = [column for column in train.columns if 'payer' in column or 'veteran' in column or 'disabled' in column or 'insured' in column]
health_status = train[health_status]

In [19]:
gender_status = [column for column in train.columns if 'male' in column or 'gender' in column]
gender_status = train[gender_status]

In [20]:
diagnosis_status = [column for column in train.columns if 'diagnosis' in column]
diagnosis_status = train[diagnosis_status]

In [21]:
air_quality = [column for column in train.columns if 'Ozone' in column or 'PM25' in column or 'N02' in column]
air_quality = train[air_quality]
air_quality

,Ozone,PM25,N02
0,52.237210,8.650555,18.606528
1,42.301121,8.487175,20.113179
2,40.108207,7.642753,14.839351
3,42.070075,7.229393,15.894123
4,41.356058,4.110749,11.722197
...,...,...,...
12897,38.753055,8.068682,21.140731
12898,36.469947,6.265266,10.728732
12899,42.070075,7.229393,15.894123
12900,37.722740,7.879795,27.496367


In [22]:
# Assuming 'your_data' is your DataFrame with target variable and one-hot encoded geographical locations
# Assuming 'target_variable' is the name of your target variable

def apply_lasso(columns):
    # Splitting data into features (X) and target variable (y)
    y = train[target]
    X = columns


    # Splitting data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



    # Standardize the Features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Define the pipeline with LASSO regression
    lasso_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('lasso', Lasso())
    ])

    # Define the parameter grid for alpha
    param_grid = {'lasso__alpha': [0.01, 0.1, 1, 10]}

    # Perform GridSearchCV
    grid_search = GridSearchCV(lasso_pipeline, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X, y)

    # Display the best parameters and results
    print("Best Parameters for LASSO:", grid_search.best_params_)
    print("Best MSE for LASSO:", -grid_search.best_score_)

    # Get the best LASSO model from the grid search
    best_lasso_model = grid_search.best_estimator_

    # Extract important features
    lasso_coefficients = best_lasso_model.named_steps['lasso'].coef_
    important_features = X.columns[lasso_coefficients != 0]

    # Display the important features
    print("Important Features:", important_features)

    return list(important_features)


In [23]:
important_ages= apply_lasso(age)

Best Parameters for LASSO: {'lasso__alpha': 0.01}
Best MSE for LASSO: 0.2338971458115935
Important Features: Index(['patient_age', 'age_40s'], dtype='object')


In [24]:
important_locs= apply_lasso(locations)

Best Parameters for LASSO: {'lasso__alpha': 0.01}
Best MSE for LASSO: 0.23355724266002936
Important Features: Index(['Region_West', 'Division_Pacific', 'patient_state_CO',
       'patient_state_IL', 'patient_state_KY', 'patient_state_MI',
       'patient_state_MN', 'patient_state_NY', 'patient_state_VA'],
      dtype='object')


In [25]:
important_race_info = apply_lasso(race)

Best Parameters for LASSO: {'lasso__alpha': 0.01}
Best MSE for LASSO: 0.23414804966709749
Important Features: Index(['race_black', 'patient_race_White'], dtype='object')


In [26]:
important_education_info = apply_lasso(education)

Best Parameters for LASSO: {'lasso__alpha': 0.01}
Best MSE for LASSO: 0.23434303904342313
Important Features: Index(['education_less_highschool', 'education_bachelors'], dtype='object')


In [27]:
important_income_info = apply_lasso(income)

Best Parameters for LASSO: {'lasso__alpha': 0.01}
Best MSE for LASSO: 0.23443895399803732
Important Features: Index(['income_household_75_to_100', 'income_individual_median'], dtype='object')


In [28]:
important_social_info = apply_lasso(social_status)

Best Parameters for LASSO: {'lasso__alpha': 0.01}
Best MSE for LASSO: 0.23448747352257215
Important Features: Index(['widowed'], dtype='object')


In [29]:
important_empl_info = apply_lasso(empl_status)

Best Parameters for LASSO: {'lasso__alpha': 0.1}
Best MSE for LASSO: 0.23448955616933126
Important Features: Index([], dtype='object')


In [30]:
important_home_info = apply_lasso(home_status)

Best Parameters for LASSO: {'lasso__alpha': 0.01}
Best MSE for LASSO: 0.2343960611312832
Important Features: Index(['home_value'], dtype='object')


In [31]:
important_health_info = apply_lasso(health_status)

Best Parameters for LASSO: {'lasso__alpha': 0.01}
Best MSE for LASSO: 0.2339278224233048
Important Features: Index(['health_uninsured', 'payer_type_COMMERCIAL', 'payer_type_MEDICAID'], dtype='object')


In [32]:
important_gender_info = apply_lasso(gender_status)

Best Parameters for LASSO: {'lasso__alpha': 0.01}
Best MSE for LASSO: 0.23448955616933126
Important Features: Index([], dtype='object')


In [33]:
important_rent_info = apply_lasso(rent)

Best Parameters for LASSO: {'lasso__alpha': 0.01}
Best MSE for LASSO: 0.23446916859507422
Important Features: Index(['rent_median'], dtype='object')


In [34]:
important_diagnosis_info = apply_lasso(diagnosis_status)

Best Parameters for LASSO: {'lasso__alpha': 0.01}
Best MSE for LASSO: 0.15033176442301516
Important Features: Index(['breast_cancer_diagnosis_code_1741',
       'breast_cancer_diagnosis_code_1742',
       'breast_cancer_diagnosis_code_1743',
       'breast_cancer_diagnosis_code_1744',
       'breast_cancer_diagnosis_code_1745',
       'breast_cancer_diagnosis_code_1746',
       'breast_cancer_diagnosis_code_1748',
       'breast_cancer_diagnosis_code_1749',
       'breast_cancer_diagnosis_code_19881',
       'breast_cancer_diagnosis_code_C50811',
       'metastatic_cancer_diagnosis_code_C7981'],
      dtype='object')


In [35]:
important_air_info = apply_lasso(air_quality)

Best Parameters for LASSO: {'lasso__alpha': 0.01}
Best MSE for LASSO: 0.23448955616933126
Important Features: Index([], dtype='object')


In [36]:
miscellaneous  = ['patient_zip3', 'population', 'density', 'poverty', 'commute_time', 'family_size', 'limited_english', 'farmer']

In [37]:
important_feats = train[
    important_health_info + 
    important_home_info + 
    important_social_info + 
    important_income_info + 
    important_education_info + 
    important_race_info + 
    important_locs + 
    important_ages + 
    important_rent_info + 
    important_diagnosis_info +
    miscellaneous + 
    list(empl_status.columns) +
    list(gender_status.columns) + 
    list(air_quality.columns)
    ]
important_feats

,health_uninsured,payer_type_COMMERCIAL,payer_type_MEDICAID,home_value,widowed,income_household_75_to_100,income_individual_median,education_less_highschool,education_bachelors,race_black,patient_race_White,Region_West,Division_Pacific,patient_state_CO,patient_state_IL,patient_state_KY,patient_state_MI,patient_state_MN,patient_state_NY,patient_state_VA,patient_age,age_40s,rent_median,breast_cancer_diagnosis_code_1741,breast_cancer_diagnosis_code_1742,breast_cancer_diagnosis_code_1743,breast_cancer_diagnosis_code_1744,breast_cancer_diagnosis_code_1745,breast_cancer_diagnosis_code_1746,breast_cancer_diagnosis_code_1748,breast_cancer_diagnosis_code_1749,breast_cancer_diagnosis_code_19881,breast_cancer_diagnosis_code_C50811,metastatic_cancer_diagnosis_code_C7981,patient_zip3,population,density,poverty,commute_time,family_size,limited_english,farmer,labor_force_participation,unemployment_rate,self_employed,male,female,patient_gender_F,Ozone,PM25,N02
0,11.200000,0.0,1.0,2.646343e+05,4.442857,12.742857,24563.57143,33.257143,8.357143,13.100000,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84,11.614286,1165.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,924,31437.75000,1189.562500,22.542857,27.814286,3.928571,10.100000,0.000000,61.528571,8.471429,13.428571,49.857143,50.142857,1.0,52.237210,8.650555,18.606528
1,7.018182,1.0,0.0,6.776885e+05,4.651515,12.469697,41287.27273,14.230303,23.739394,2.527273,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62,13.333333,2003.125000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,928,39121.87879,2295.939394,10.109091,30.606061,3.622727,8.057576,0.027273,65.230303,5.103030,15.224242,49.893939,50.106061,1.0,42.301121,8.487175,20.113179
2,15.066667,1.0,0.0,2.377131e+05,4.710000,13.560000,40399.03333,10.811667,19.678333,9.231667,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43,12.860000,1235.907407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,760,21996.68333,626.236667,9.663333,31.394915,3.260667,3.356667,3.650847,66.428333,4.560000,13.722034,50.123333,49.876667,1.0,40.108207,7.642753,14.839351
3,4.404762,1.0,0.0,1.012474e+06,4.623810,10.721429,55336.28571,5.835714,33.285714,1.438095,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45,12.650000,2354.738095,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,926,32795.32558,1896.220930,8.688095,27.561905,3.098095,5.280952,0.052381,64.430952,5.264286,18.502381,49.066667,50.933333,1.0,42.070075,7.229393,15.894123
4,12.088000,1.0,0.0,2.498457e+05,5.406000,12.480000,29073.18367,11.576000,13.978000,0.426000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55,12.600000,919.743590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,836,10886.26000,116.886000,11.224000,26.170213,3.352653,1.946000,6.890909,57.488000,4.258000,13.029545,52.312000,47.688000,1.0,41.356058,4.110749,11.722197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12897,6.300000,0.0,0.0,1.005470e+05,5.705882,11.047059,27888.52941,12.923529,14.400000,27.770588,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50,11.694118,772.647059,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,436,19413.05882,1196.805882,23.600000,19.841176,3.039412,0.864706,0.023529,61.429412,9.135294,9.105882,48.264706,51.735294,1.0,38.753055,8.068682,21.140731
12898,3.845679,1.0,0.0,8.179491e+05,5.466667,11.286250,52778.65000,8.916049,26.903704,5.906173,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50,12.817284,2223.445946,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,945,30153.87952,976.289157,7.837037,34.700000,3.271125,5.411250,0.416250,63.281481,5.332099,14.116250,49.727160,50.272840,1.0,36.469947,6.265266,10.728732
12899,4.404762,1.0,0.0,1.012474e+06,4.623810,10.721429,55336.28571,5.835714,33.285714,1.438095,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61,12.650000,2354.738095,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,926,32795.32558,1896.220930,8.688095,27.561905,3.098095,5.280952,0.052381,64.430952,5.264286,18.502381,49.066667,50.93333

In [38]:
important_feats.to_csv('imp_train.csv', index=False)

In [39]:
train_2 = pd.read_csv('train_2.csv')

In [43]:
# Splitting data into features (X) and target variable (y)
y = train[target]
X = train_2


# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

In [44]:
# Creating a pipeline with different classifiers and scalers
pipelines = {
    'RandomForest': Pipeline([
        ('scaler', StandardScaler(with_mean=False)),
        ('classifier', RandomForestClassifier(random_state=42))
    ]),
    
    'SVM': Pipeline([
        ('scaler', MinMaxScaler()),
        ('classifier', SVC(random_state=42))
    ]),
    
    'KNN': Pipeline([
        ('scaler', StandardScaler(with_mean=False)),
        ('classifier', KNeighborsClassifier())
    ])
}

# Define the parameter grids for each classifier
param_grid = {
    'RandomForest': {
        'classifier__n_estimators': [50, 100, 200],
        'classifier__max_depth': [None, 10, 20],
    },
    'SVM': {
        'classifier__C': [0.1, 1, 10],
        'classifier__gamma': [0.1, 0.01, 0.001],
    },
    'KNN': {
        'classifier__n_neighbors': [3, 5, 7],
        'classifier__weights': ['uniform', 'distance'],
    }
}

# Dictionary to store feature importances for each model
feature_importances_dict = {}

# Training and evaluating each model with GridSearchCV
for name, pipeline in pipelines.items():
    grid_search = GridSearchCV(pipeline, param_grid=param_grid[name], cv=5, scoring='accuracy', n_jobs=-1)
    
    print(f"\nTraining {name} model with GridSearchCV...")
    grid_search.fit(X_train, y_train)
    
    # Display the best parameters and results
    print(f"Best Parameters for {name}:", grid_search.best_params_)
    print(f"Best Accuracy for {name}:", grid_search.best_score_)

    # Get the best model from the grid search
    best_model = grid_search.best_estimator_

    # Extract feature importances if the model is RandomForestClassifier
    if name == 'RandomForest':
        random_forest_model = best_model.named_steps['classifier']
        feature_importances_dict[name] = dict(zip(X_train.columns, random_forest_model.feature_importances_))
    else:
        feature_importances_dict[name] = None

    # Evaluate the best model on the test set
    y_pred = best_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    print(f"\nBest {name} Model Evaluation:\n{accuracy}\n{report}")


Training RandomForest model with GridSearchCV...
Best Parameters for RandomForest: {'classifier__max_depth': 10, 'classifier__n_estimators': 100}
Best Accuracy for RandomForest: 0.8092240084839613

Best RandomForest Model Evaluation:
0.7996900426191399
              precision    recall  f1-score   support

           0       0.90      0.52      0.66       954
           1       0.77      0.96      0.86      1627

    accuracy                           0.80      2581
   macro avg       0.83      0.74      0.76      2581
weighted avg       0.82      0.80      0.78      2581


Training SVM model with GridSearchCV...
Best Parameters for SVM: {'classifier__C': 1, 'classifier__gamma': 0.1}
Best Accuracy for SVM: 0.8131965482290671

Best SVM Model Evaluation:
0.8039519566059666
              precision    recall  f1-score   support

           0       0.90      0.53      0.67       954
           1       0.78      0.96      0.86      1627

    accuracy                           0.80      2581

In [45]:
# Display feature importances
for name, feature_importances in feature_importances_dict.items():
    print(f"\nFeature Importances for {name} model:")
    if feature_importances:
        for feature, importance in sorted(feature_importances.items(), key=lambda x: x[1], reverse=True):
            print(f"{feature}: {importance}")
    else:
        print(f"No feature importances available for {name} model.")


Feature Importances for RandomForest model:
breast_cancer_diagnosis_code_1749: 0.46573490809185186
breast_cancer_diagnosis_code_1744: 0.12011567650832358
breast_cancer_diagnosis_code_1748: 0.07340567618387302
patient_age: 0.059611509347532024
breast_cancer_diagnosis_code_1741: 0.02451095758264519
breast_cancer_diagnosis_code_1742: 0.01957172518221665
breast_cancer_diagnosis_code_1745: 0.01610940597804147
metastatic_cancer_diagnosis_code_C7981: 0.015153974960266846
age_40s: 0.010139664664096335
health_uninsured: 0.00997506820550184
patient_zip3: 0.00994233962209006
self_employed: 0.009610246422567513
commute_time: 0.00945002803036941
home_value: 0.009150776385944288
Ozone: 0.009014983466647397
labor_force_participation: 0.008788225743832671
income_individual_median: 0.008614827168987142
income_household_75_to_100: 0.008525295063668562
PM25: 0.00835133063360677
education_less_highschool: 0.0080884015302378
race_black: 0.008057839701680012
family_size: 0.007914672852897344
N02: 0.0074972